<a href="https://colab.research.google.com/github/embedashishg/SusAgri/blob/SusAgri/VLC_Selecton_GAppScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The text below is to be copied to a AppScript linked to Google Sheets.


In [ ]:

function updateApplicantsSheet() {
  try {

    var destinationSpreadsheet = SpreadsheetApp.getActiveSpreadsheet();
    var configSheet = destinationSpreadsheet.getSheetByName("Configurations");
    var c8Value = configSheet.getRange("C8").getValue().toString().toLowerCase();
    if (c8Value === "yes") {
      var sourceSheetId = configSheet.getRange("C5").getValue();
      var sourceSpreadsheet = SpreadsheetApp.openById(sourceSheetId);

      // Get the name of the first sheet in the source spreadsheet
      var sourceSheetFirst = sourceSpreadsheet.getSheets()[0];
      var sourceSheet = sourceSheetFirst;

      var data = sourceSheet.getDataRange().getValues();
      data.sort(function (a, b) {
        if (b[2] === a[2]) {
          return a[0] - b[0];
        }
        return b[2] - a[2];
      });

      var applicantsSheet = destinationSpreadsheet.getSheetByName("Applicants");
      applicantsSheet.clear();
      var finalParticipantsSheet = destinationSpreadsheet.getSheetByName("FinalParticipants");
      finalParticipantsSheet.clear();

      // Determine the Cut-Off Score from cell C6 in 'Configurations' sheet
      var cutOffScore = configSheet.getRange("C6").getValue();

      // Determine the number of participants to select from cell C7 in 'Configurations' sheet
      var numberOfParticipants = configSheet.getRange("C7").getValue();

      // Select the top participants based on the configured count and score
      var finalParticipantsData = data.filter(function (applicant) {
        return applicant[2] >= cutOffScore;
      }).slice(0, numberOfParticipants);

      // Write the selected data to the 'Applicants' sheet
      if (data.length > 0) {
        applicantsSheet.getRange(1, 1, data.length, data[0].length).setValues(data);
      }

      // Log a message to indicate that the update is complete
      Logger.log("Data has been updated in the 'Applicants' sheet.");

      // Check if there are final participants to write to 'FinalParticipants' sheet
      if (finalParticipantsData.length > 0) {
        // Write the selected data to the 'FinalParticipants' sheet, starting from the 2nd row
        finalParticipantsSheet.getRange(2, 1, finalParticipantsData.length, finalParticipantsData[0].length).setValues(finalParticipantsData);

        // Get the header row from the 'Applicants' sheet
        var headerRow = applicantsSheet.getRange(1, 1, 1, applicantsSheet.getLastColumn()).getValues();

        // Place the header row in the 'FinalParticipants' sheet
        finalParticipantsSheet.getRange(1, 1, 1, headerRow[0].length).setValues(headerRow);

        Logger.log("Data has been updated in the 'FinalParticipants' sheet.");
      } else {
        Logger.log("No final participants to update in the 'FinalParticipants' sheet.");
      }
    } else if (c8Value === "no") {
      // If C8 is "NO," clear both 'Applicants' and 'FinalParticipants' sheets
      var applicantsSheet = destinationSpreadsheet.getSheetByName("Applicants");
      var finalParticipantsSheet = destinationSpreadsheet.getSheetByName("FinalParticipants");

      applicantsSheet.clear();
      finalParticipantsSheet.clear();
      Logger.log("Cleared data in 'Applicants' and 'FinalParticipants' sheets.");
    } else {
      // Handle other values of C8 as needed
      Logger.log("C8 has an unsupported value.");
    }
  } catch (error) {
    Logger.log("Error: " + error.toString());
  }
}